<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [3]:
!pip3 install beautifulsoup4
!pip3 install requests

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    #return out
    #strings = [s.strip() for s in table_cells.strings if s.strip()]
   # out = ''.join([s for i, s in enumerate(strings) if i % 2 == 0])
    #return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    #out=[i for i in table_cells.strings][0]
    #return out
    strings = [s.strip() for s in table_cells.strings if s.strip()]
    return strings[0] if strings else "N/A"


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    #if mass:
        #mass.find("kg")
        #new_mass=mass[0:mass.find("kg")+2]
    #else:
        #new_mass=0
    #return new_mass
    if "kg" in mass:
        new_mass = mass[:mass.find("kg")+2]
    else:
        new_mass = "0"
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    #colunm_name = ' '.join(row.contents)
    colunm_name = ' '.join([str(c) for c in row.contents])

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name
print('hello')

hello


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}


#  added

#soup = BeautifulSoup(resp.text, 'html.parser')
#print('Fetched page:', static_url)

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
resp = requests.get(static_url, headers=headers, timeout=20)
print(resp.raise_for_status())
print("Status code:", resp.status_code)

None
Status code: 200


Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(resp.text, 'html.parser')
print('Fetched page:', static_url)

Fetched page: https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
#html_tables = soup.find_all('table', class_='wikitable plainrowheaders collapsible')
#html_tables = soup.find('table', class_='wikitable plainrowheaders collapsible')
html_tables = soup.find_all("table")

print('Found', len(html_tables), 'tables')

Found 25 tables


Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
#column_names = ["Flight Number","Date","Booster Version","Launch Site","Payload","Payload Mass","Orbit","Customer","Launch outcome","Booster landing"]
column_names =[]
#for th in first_launch_table.find_all('th'):
 #   col_name = extract_column_from_header(th)
  #  column_names.append(col_name)
element = soup.find_all('th')
for row in range(len(element)):
    try:
        name = extract_column_from_header(element[row])
        if (name is not None and len(name) > 0):
            column_names.append(name)
    except:
        pass
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [12]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'N/A', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'FH 2', 'FH 3', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight\n <p>No.\n</p>', 'Date and time ( )', 'L

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [13]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [14]:
# Starting from the third table (index 2), which contains actual launch records
launch_dict = {
    "Flight No.": [],
    "Date": [],
    "Time (UTC)": [],
    "Version Booster": [],
    "Launch Site": [],
    "Payload": [],
    "Payload Mass": [],
    "Orbit": [],
    "Customer": [],
    "Launch Outcome": [],
    "Landing Outcome": []
}

for table in html_tables[2:]:  # skip the first two tables
    rows = table.find_all("tr")
    
    for row in rows[1:]:  # skip header
        cells = row.find_all("td")
        if not cells:  # skip empty rows
            continue
        
        # Use safe get with default
        def get_cell(idx):
            return cells[idx].text.strip() if idx < len(cells) else "N/A"
        
        # Flight No.
        launch_dict["Flight No."].append(get_cell(0))
        
        # Date and Time
        if len(cells) > 1:
            dt = date_time(cells[1])
            launch_dict["Date"].append(dt[0] if len(dt) > 0 else "N/A")
            launch_dict["Time (UTC)"].append(dt[1] if len(dt) > 1 else "N/A")
        else:
            launch_dict["Date"].append("N/A")
            launch_dict["Time (UTC)"].append("N/A")
        
        # Version Booster
        launch_dict["Version Booster"].append(booster_version(cells[2]) if len(cells) > 2 else "N/A")
        
        # Launch Site
        launch_dict["Launch Site"].append(get_cell(3))
        
        # Payload
        launch_dict["Payload"].append(get_cell(4))
        
        # Payload Mass
        launch_dict["Payload Mass"].append(get_mass(cells[5]) if len(cells) > 5 else "0")
        
        # Orbit
        launch_dict["Orbit"].append(get_cell(6))
        
        # Customer
        launch_dict["Customer"].append(get_cell(7))
        
        # Launch Outcome
        launch_dict["Launch Outcome"].append(get_cell(8))
        
        # Landing Outcome
        launch_dict["Landing Outcome"].append(
            landing_status(cells[9]) if len(cells) > 9 else "N/A"
        )


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [18]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
#df.to_csv('spacex_web_scraped.csv', index=False)
df.head(5)
geo_launches = df[df['Orbit'].isin(['GTO', 'GEO'])]

# Count them
count_geo = geo_launches.shape[0]
print(count_geo)
print(df['Orbit'].unique())
geo_count = df[df['Orbit'].isin(['GTO', 'GEO'])].shape[0]

print("Number of launches to geosynchronous orbit:", geo_count)
#df.to_csv('spacex_web_scraped.csv', index=False)

0
['SpaceX' 'N/A' 'NASA (COTS)\nNRO' 'NASA (COTS)' 'NASA (CRS)' 'MDA' 'SES'
 'Thaicom' 'Orbcomm' 'AsiaSat' 'USAF\nNASA\nNOAA' 'ABS\nEutelsat'
 'Turkmenistan NationalSpace Agency[88]' 'NASA (LSP)\nNOAA\nCNES'
 'SKY Perfect JSAT Group' 'Spacecom' 'Iridium Communications' 'EchoStar'
 'NRO' 'Inmarsat' 'Bulsatcom' 'Intelsat' 'NSPO' 'USAF'
 'SES S.A.\nEchoStar' 'KT Corporation' 'Northrop Grumman [f][238]'
 'Hisdesat\nexactEarth\nSpaceX' 'Hispasat[277]\nNovaWurks' 'NASA (LSP)'
 'Thales-Alenia / BTRC' 'Iridium Communications\nGFZ\xa0• NASA' 'Telesat'
 'Telkom Indonesia' 'CONAE' "Es'hailSat" 'Spaceflight Industries'
 'PSN\nSpaceIL / IAI\nAir Force Research' 'NASA (CCD)' 'Arabsat'
 'Canadian Space Agency (CSA)' 'Sky Perfect JSATKacific 1'
 'NASA (CTS)[497]' 'NASA (CCDev)' 'SpaceXPlanet Labs'
 'U.S. Space Force[530]' 'Republic of Korea Army'
 'SpaceXSpaceflight Industries (BlackSky)' 'CONAEPlanetIQTyvak' 'USSF'
 'NASA (CCP)[497]' 'NASA / NOAA / ESA / EUMETSAT' 'Sirius XM' 'Türksat'
 'Various' 'Sp

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
